<a href="https://colab.research.google.com/github/hei1sme/CPV301-group-3/blob/main/WORKSHOP/workshop_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 👉 1. Cài đặt thư viện nếu cần
!pip install opencv-python-headless scikit-image scikit-learn matplotlib --quiet

# 👉 2. Import thư viện
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import gaussian
from skimage.segmentation import active_contour
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth
from google.colab import files

# 👉 3. Upload ảnh từ máy
uploaded = files.upload()
filename = list(uploaded.keys())[0]
img = cv2.imread(filename)

# 👉 4. Hàm Snakes (Active Contour)
def run_snakes(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = gray / 255.0
    gray_smooth = gaussian(gray, 3)

    s = np.linspace(0, 2*np.pi, 400)
    x = 150 + 100*np.cos(s)
    y = 150 + 100*np.sin(s)
    init = np.array([x, y]).T

    snake = active_contour(gray_smooth, init, alpha=0.015, beta=10, gamma=0.001)

    plt.figure(figsize=(8,6))
    plt.imshow(gray, cmap='gray')
    plt.plot(init[:, 0], init[:, 1], '--r', label='Initial')
    plt.plot(snake[:, 0], snake[:, 1], '-b', label='Snakes')
    plt.title("Snakes Algorithm")
    plt.legend()
    plt.axis('off')
    plt.show()

# 👉 5. Hàm Watershed
def run_watershed(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    markers = cv2.watershed(img, markers)
    img_ws = img.copy()
    img_ws[markers == -1] = [255, 0, 0]

    plt.figure(figsize=(8,6))
    plt.imshow(cv2.cvtColor(img_ws, cv2.COLOR_BGR2RGB))
    plt.title("Watershed Segmentation")
    plt.axis('off')
    plt.show()

# 👉 6. Hàm K-Means (đã fix lỗi)
def run_kmeans(img, k=3):
    Z = img.reshape((-1, 3))
    Z = np.float32(Z)

    kmeans = KMeans(n_clusters=k, random_state=0, n_init=10).fit(Z)
    centers = np.uint8(kmeans.cluster_centers_)
    labels = kmeans.labels_

    segmented = centers[labels.flatten()]
    segmented_image = segmented.reshape(img.shape)

    plt.figure(figsize=(8,6))
    plt.imshow(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))
    plt.title(f"K-Means Segmentation (k={k})")
    plt.axis('off')
    plt.show()

# 👉 7. Hàm Mean Shift
def run_meanshift(img):
    flat_img = img.reshape((-1, 3))
    flat_img = np.float32(flat_img)

    bandwidth = estimate_bandwidth(flat_img, quantile=0.1, n_samples=500)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(flat_img)

    labels = ms.labels_
    centers = np.uint8(ms.cluster_centers_)
    segmented = centers[labels.flatten()]
    segmented_image = segmented.reshape(img.shape)

    plt.figure(figsize=(8,6))
    plt.imshow(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))
    plt.title("Mean Shift Segmentation")
    plt.axis('off')
    plt.show()

# 👉 8. Menu điều khiển chính
while True:
    print("\n🎯 Chọn thuật toán segmentation:")
    print("1. Snakes (Active Contour)")
    print("2. Watershed")
    print("3. K-Means")
    print("4. Mean Shift")
    print("0. Thoát")

    choice = input("Nhập lựa chọn của bạn (0-4): ")

    if choice == '1':
        run_snakes(img)
    elif choice == '2':
        run_watershed(img)
    elif choice == '3':
        try:
            k = int(input("Nhập số cụm (k): "))
            run_kmeans(img, k)
        except ValueError:
            print("⚠️ Vui lòng nhập số nguyên hợp lệ cho k.")
    elif choice == '4':
        run_meanshift(img)
    elif choice == '0':
        print("✅ Đã thoát chương trình.")
        break
    else:
        print("⚠️ Vui lòng nhập đúng số từ 0 đến 4.")


IndexError: list index out of range